This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# 自然語言處理任務：閱讀理解

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
1. `工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`
2. `編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`

## 練習題：國文考試小幫手

都當過學生的我們，以前在國文考試的時候必定是頭痛到不行，尤其是遇到閱讀理解測驗題型的時候，不只要去看落落長的文章，還要根據問題去猜測答案，實在太強人所難了。現在我們要運用課堂所學，訓練一個閱讀理解的模型，讓機器代替我們回答這些困難的問題吧！

訓練資料為 [DRCD](https://github.com/DRCKnowledgeTeam/DRCD)（台達閱讀理解資料集），我們從中擷取訓練資料 1,000 筆，測試資料 2,00 筆。

![](https://i.imgur.com/pDWnlFq.png)

（[圖片來源](https://memes.tw/wtf/388869)）

In [ ]:
!pip install numpy pandas torch simpletransformers

In [ ]:
import json

import numpy as np
from simpletransformers.question_answering import QuestionAnsweringModel

### 資料準備

首先我們先下載訓練資料（`drcd_train.json`）與測試資料（`drcd_test.json`）到 Colab Server 的硬碟裡。

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/DRCD/drcd_train.json -O drcd_train.json

In [ ]:
!wget https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/DRCD/drcd_test.json -O drcd_test.json

由於資料是 json 檔，我們可以需要用 `json` 這個套件來讀取。讀出來的資料為一個 list，list 的一個元素代表一筆訓練（測試）資料，資料型態為 dict，範例如下：

```json
{
    "context": "根據研究結果，特斯拉從Panasonic、LG以及寧德時代這三家供應商處購買電池，每千瓦∕小時的電池平均價格為142美元。與此同時，諸如通用汽車這樣的公司，每千瓦∕小時的電池平均價格為169美元，而產業平均價格則落在186美元左右，也就是說，特斯拉的採購價要比產業平均水準要低23%。然而，特斯拉每千瓦∕小時的電池價格是如何比競爭對手低這麼多？凱恩能源表示，這是因為特斯拉執行長伊隆．馬斯克和研發工程團隊在盡可能地削減汽車成本，這是特斯拉整體規劃中的關鍵戰略。",
    "qas": [{
        "id": "0",
        "question": "特斯拉的執行長是誰？",
        "answers": [{
            "text": "伊隆．馬斯克",
            "answer_start": 189,
        }],
    }],
}
```

這種格式原始為 [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) 格式，並且做稍微修改，讓接下來的模型可以認得的資料格式。

In [ ]:
train_data = json.loads(open("drcd_train.json").read())
test_data = json.loads(open("drcd_test.json").read())

這裡來查看一下資料的分佈狀況。

In [ ]:
print("Number of training data: {}".format(len(train_data)))
print("Number of testing data: {}".format(len(test_data)))

### 模型訓練

這裡我們要用 [Simple Transformers](https://github.com/ThilinaRajapakse/simpletransformers) 的 [Question Answering](https://simpletransformers.ai/docs/qa-specifics/) 來建立模型、訓練以及測試。

模型的架構我們使用 BERT，是由 CKIP Lab 針對中文文本做過預訓練的，我們要用它來做下游任務的 fine-tuning。相關的參數請參考[這裡](https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model)。

In [ ]:
# Create a question answering model based on the pre-trained BERT.
model = QuestionAnsweringModel(
    "bert",
    "ckiplab/bert-base-chinese",
    args={
        "learning_rate": 0.0001,
        "num_train_epochs": 10,
        "train_batch_size": 32,
        "overwrite_output_dir": True,
    },
)

訓練模型非常簡單，只要呼叫 `train_model()`，等待結果即可。

In [ ]:
# Train the model.
model.train_model(train_data)

做 evaluation 也非常簡單，只要呼叫 `eval_model()` 就會把對測試資料相關的 evaluation metrics 等資訊計算出來，我們可以從中得到完全正確（`correct`）、部份正確（`similar`）以及不正確（`incorrect`）的數量，並且算出比例。

關於閱讀理解常用的 evaluation metrics，更正式的會使用 Exact Match（EM）、F1 Score 等，詳細請參考[這篇文章](https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html#Metrics-for-QA)。

In [ ]:
# Evaluate the model.
result, texts = model.eval_model(test_data)
print(result)

In [ ]:
num_total = (result["correct"] + result["similar"] + result["incorrect"])
correct_percent = result["correct"] / num_total
similar_percent = result["similar"] / num_total
incorrect_percent = result["incorrect"] / num_total

print("Correct Percent: {}".format(correct_percent))
print("Similar Percent: {}".format(similar_percent))
print("Incorrect Percent: {}".format(incorrect_percent))

如果只是要單純的拿到預測結果，呼叫 `predict()` 即可。

In [ ]:
to_predict = [{
    "context": "在歐洲，梵語的學術研究，由德國學者陸特和漢斯雷頓開創。後來威廉·瓊斯發現印歐語系，也要歸功於對梵語的研究。此外，梵語研究，也對西方文字學及歷史語言學的發展，貢獻不少。1786年2月2日，亞洲協會在加爾各答舉行。會中，威廉·瓊斯發表了下面這段著名的言論：「梵語儘管非常古老，構造卻精妙絕倫：比希臘語還完美，比拉丁語還豐富，精緻之處同時勝過此兩者，但在動詞詞根和語法形式上，又跟此兩者無比相似，不可能是巧合的結果。這三種語言太相似了，使任何同時稽考三者的語文學家都不得不相信三者同出一源，出自一種可能已經消逝的語言。基於相似的原因，儘管缺少同樣有力的證據，我們可以推想哥德語和凱爾特語，雖然混入了迥然不同的語彙，也與梵語有著相同的起源；而古波斯語可能也是這一語系的子裔。",
    "qas": [{
        "id": "0",
        "question": "陸特和漢斯雷頓開創了哪一地區對梵語的學術研究？",
        "answers": [{
            "text": "歐洲",
            "answer_start": 1,
        }],
    }],
}]

# Make predictions with the model.
predictions, raw_outputs = model.predict(to_predict)
print(predictions)

In [ ]:
to_predict = [{
    "context": "鮭魚肉含有高蛋白質、歐媒咖-3脂肪酸及抗佝僂病維他命，被視為有益的食物。此外，膽固醇含量也比較高，達每100克肉就有214微克的膽固醇。鮭魚肉一般呈橙色或深紅色也有很多白肉的品種，野生鮭靠蝦子等甲殼類維生，魚肉才會呈現橘紅色，在大西洋鮭養殖業者中為了讓鮭魚類似野生的蝦紅素表現，在飼料中添加類胡蘿蔔素等可食用色素來增色提加賣相。接近99%在大西洋出產的鮭魚為人工飼養。此外轉基因鮭魚2015年在美國被核准上市，可能成為了第一種上餐桌的轉基因肉品，這種鮭魚只要三分之一的養殖時間。鮭魚的食法有多種，在日本，烤鮭魚是家常菜。日本人還會把鮭魚頭製成鹽燒鮭魚等菜式；歐洲及美國人則會以熱或冷煙燻方式製作煙燻鮭魚，或把鮭魚製成罐頭以便儲存。此外，北歐地區亦有醃製鮭魚。此外，挪威為了開拓日本的鮭魚市場，發明了鮭魚壽司。",
    "qas": [{
        "id": "0",
        "question": "什麼維他命可以經鮭魚肉攝取到？",
        "answers": [{
            "text": "抗佝僂病維他命",
            "answer_start": 19,
        }],
    }],
}]

# Make predictions with the model.
predictions, raw_outputs = model.predict(to_predict)
print(predictions)

練習時間：

我們想要讓模型的精準度上升，除了更改模型的架構或是參數之外，我們也可以增加更多的訓練資料進去。原本的訓練資料有 1,000 筆，現在我們有一個更多訓練資料的版本，總共有 3,000 筆（[連結](https://raw.githubusercontent.com/SuJiaKuan/fgu_ai_course/main/datasets/nlp_tw/DRCD/drcd_train_long.json)），請試著用新版本的資料來做訓練，並請看看準確率是否有上升，並說明可能的原因。